In [ ]:
pip install -q transformers trl peft bitsandbytes accelerate

In [1]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

# 1. YOUR DATA
# Define your Q&A pairs here. The more you have, the better.
data =[
    {
        "text": "<|user|>\nWho is Sidharth E?\n<|assistant|>\nSidharth E is a Full Stack Developer based in Kerala, India, with experience in building scalable web applications. He specializes in React, Node.js, cloud platforms (Azure/AWS), and AI integrations."
    },
    {
        "text": "<|user|>\nWhat is Sidharth's current job title?\n<|assistant|>\nHe is currently a Full Stack Developer at KellyOCG."
    },
    {
        "text": "<|user|>\nHow many years of experience does Sidharth have?\n<|assistant|>\nSidharth has over 4 years of experience. He started his career in January 2020 at Tata Consultancy Services."
    },
    {
        "text": "<|user|>\nWhat are Sidharth's core frontend skills?\n<|assistant|>\nHis frontend expertise includes HTML5/CSS/JS, TypeScript, React, Next.js, SASS, and Tailwind CSS."
    },
    {
        "text": "<|user|>\nWhich backend technologies does Sidharth use?\n<|assistant|>\nHe is proficient in Node.js, ExpressJS, Python, and FastAPI."
    },
    {
        "text": "<|user|>\nDoes Sidharth have experience with Cloud platforms?\n<|assistant|>\nYes, he has extensive experience with Azure and AWS. He also uses tools like Jenkins, Nginx, Bicep, and Linux for DevOps and CI/CD pipelines."
    },
    {
        "text": "<|user|>\nWhat AI and Machine Learning technologies does he know?\n<|assistant|>\nSidharth works with Agentic AI, Langchain, HuggingFace, MCP, GenAI, OpenAI, Gemini, and Azure AI."
    },
    {
        "text": "<|user|>\nWhat database systems is he familiar with?\n<|assistant|>\nHe uses MongoDB and CosmosDB."
    },
    {
        "text": "<|user|>\nWhat development tools does Sidharth use?\n<|assistant|>\nHe utilizes Git, Postman, VS Code, Cursor, Windsurf, Antigravity, and Microsoft Office tools like Excel and PowerPoint."
    },
    {
        "text": "<|user|>\nHow can I contact Sidharth E?\n<|assistant|>\nYou can email him at sidharthe38943@gmail.com or call him at +91 6238471816. His portfolio is available at https://sidharthe.dev."
    },
    {
        "text": "<|user|>\nTell me about his work at KellyOCG.\n<|assistant|>\nAt KellyOCG (Aug 2024 – Present), Sidharth develops AI-powered browser extensions. Key projects include 'GRACE Sync' and 'Project GRACE 2.0', focusing on Azure, OpenAI, and Cosmos DB integrations."
    },
    {
        "text": "<|user|>\nWhat is GRACE Sync?\n<|assistant|>\nGRACE Sync is an AI-powered browser extension developed by Sidharth. It processes data from internal web apps, features an AI chatbot, and uses Azure Identity for security. It also includes telemetry via PowerBI and history management with Cosmos DB."
    },
    {
        "text": "<|user|>\nWhat features did he implement in Project GRACE 2.0?\n<|assistant|>\nFor GRACE 2.0, he integrated file uploads with Gemini, added OCR for images, implemented real-time web search via Perplexity API, ensured Veracode compliance, and added Role-Based Access Control (RBAC)."
    },
    {
        "text": "<|user|>\nWhat did Sidharth do at Poumki Digital LLP?\n<|assistant|>\nFrom April 2023 to June 2024, he worked as a Full Stack Developer. He built the 'Project Convergence' data analytics platform, the 'e-slide' presentation platform, and enhanced the 'Izycourse' platform."
    },
    {
        "text": "<|user|>\nDescribe Project Convergence.\n<|assistant|>\nProject Convergence is a Data Analytics Platform built on the MERN stack. Sidharth implemented CI/CD with Jenkins on AWS, set up a rollback system, reduced storage costs with Nexus/S3, and automated MongoDB backups."
    },
    {
        "text": "<|user|>\nWhat is Project e-slide?\n<|assistant|>\nIt is a presentation sharing platform where Sidharth added multilingual support using React-i18next and implemented user safety features like banning capabilities. The tech stack included React, SASS, DynamoDB, and AWS Lambda."
    },
    {
        "text": "<|user|>\nWhat was Sidharth's role at TCS?\n<|assistant|>\nHe held two roles at TCS: Site Reliability Engineer (Jan 2020 – Mar 2021) and Full Stack Developer for Rapid Labs (Mar 2021 – Apr 2023)."
    },
    {
        "text": "<|user|>\nDid Sidharth work on healthcare projects?\n<|assistant|>\nYes, at TCS Rapid Labs, he built AI/ML prototypes for healthcare, including an ulcer detection model with 92% precision and a ticket prediction model with 85% accuracy using Python and Flask."
    },
    {
        "text": "<|user|>\nWhat automation tools did he build as an SRE?\n<|assistant|>\nHe built a 'Core Analysis Tool' to reduce incident resolution time by 30%, an 'Akamai report generator' saving 200+ hours annually, and an 'Auto Delegation Tool' to validate data discrepancies."
    },
    {
        "text": "<|user|>\nWhat is Sidharth's educational background?\n<|assistant|>\nHe is currently pursuing a Master of Computer Applications (MCA) in AI/ML at Amritha Vishwa Vidyapeetham (since Oct 2025). He holds a Bachelor of Computer Applications (BCA) from VLB Janakiammal College of Arts and Science (2017–2020)."
    },
    {
        "text": "<|user|>\nHas Sidharth received any awards?\n<|assistant|>\nYes, he received the 'Star of the Month' award at TCS for developing three automation tools that saved over 500 work hours annually."
    },
    {
        "text": "<|user|>\nWhat languages does Sidharth speak?\n<|assistant|>\nHe speaks Malayalam (Native), English (Professional Proficiency), Tamil (Conversational), and Hindi (Conversational)."
    },
    {
        "text": "<|user|>\nDoes Sidharth know how to use Docker or Containers?\n<|assistant|>\nHis skills list includes Linux and CI/CD pipelines. While Docker isn't explicitly listed in the 'Tools' array, his experience with AWS EC2, Jenkins, and Nginx suggests familiarity with containerized environments."
    },
    {
        "text": "<|user|>\nIs Sidharth familiar with secure coding practices?\n<|assistant|>\nYes, in Project GRACE 2.0, he achieved full Veracode compliance by fixing high-severity vulnerabilities and implemented RBAC (Role-Based Access Control) for security."
    },
    {
        "text": "<|user|>\nDoes he have experience with Search APIs?\n<|assistant|>\nYes, he implemented real-time web search using the Perplexity API for Project GRACE 2.0."
    },
    {
        "text": "<|user|>\nWhat specific AI models has he integrated?\n<|assistant|>\nHe has integrated OpenAI models and Google Gemini. He also lists 'HuggingFace' and 'Langchain' in his AI/ML skills."
    },
    {
        "text": "<|user|>\nWhat is the 'Antigravity' tool mentioned in his skills?\n<|assistant|>\nAntigravity is listed under his 'Tools' category, likely referring to an AI-assisted coding or development tool he utilizes."
    },
    {
        "text": "<|user|>\nCan Sidharth build browser extensions?\n<|assistant|>\nYes, he built 'GRACE Sync,' a complex browser extension that reinjects data into web apps and manages chat history."
    },
    {
        "text": "<|user|>\nWhat cloud infrastructure as code (IaC) tools does he use?\n<|assistant|>\nHe uses Bicep, specifically mentioned in the tech stack for Project GRACE 2.0 on Azure."
    },
    {
        "text": "<|user|>\nDoes Sidharth have experience with real-time communication?\n<|assistant|>\nYes, he implemented real-time user interactions using Socket.IO for Project Convergence and lists WebSockets under his 'Other' skills."
    },
    {
        "text": "<|user|>\nWhat authentication methods has he implemented?\n<|assistant|>\nHe has experience with JWT (JSON Web Tokens) for REST APIs and Azure Identity Service for securing browser extensions."
    },
    {
        "text": "<|user|>\nWhere is Sidharth located?\n<|assistant|>\nHe is located in Kerala, India."
    },
    {
        "text": "<|user|>\nWhat frameworks did he use for the Healthcare prototypes?\n<|assistant|>\nHe used AngularJS, Python, Django, Vue.js, and Flask."
    },
    {
        "text": "<|user|>\nWhat was the impact of the Akamai report generator?\n<|assistant|>\nIt automated report generation, saving over 200 work hours annually at TCS."
    },
    {
        "text": "<|user|>\nDoes Sidharth do UI/UX design?\n<|assistant|>\nYes, he mentions modernizing UI/UX design for GRACE Sync, optimizing mobile experiences, and performing UI enhancements for Izycourse."
    }
]

# Convert to Hugging Face Dataset format
dataset = Dataset.from_list(data)

# 2. LOAD DEEPSEEK R1 DISTILL (1.5B)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Quantization config (Makes the model smaller to fit in memory)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Fix for padding issues

# 3. CONFIGURE TRAINING (LoRA)
# This tells the system to only train a small "adapter" layer, not the whole massive model
peft_config = LoraConfig(
    r=16,       # Rank (complexity of the update matrices)
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# 4. START TRAINING
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text", # The column name in your data
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        logging_steps=1,
        num_train_epochs=3, # Run over the data 3 times
        fp16=True,
    ),
    peft_config=peft_config,
)

print("Starting training...")
trainer.train()

# 5. SAVE THE NEW MODEL
new_model_name = "Sidharth-Portfolio-Bot"
trainer.model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)
print(f"Model saved to {new_model_name}")

c:\Users\sidha\Desktop\Projects\Mind Train\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: Could not import module 'validate_bnb_backend_availability'. Are this object's requirements defined correctly?

In [ ]:
from peft import PeftModel

# Load the base model again
base_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Merge your trained "adapter" into the base model
model = PeftModel.from_pretrained(base_model, "Sidharth-Portfolio-Bot")
model = model.merge_and_unload() # Fuses the weights for faster speed

# Inference Function
def ask_bot(question):
    prompt = f"<|user|>\n{question}\n<|assistant|>\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=50, 
        do_sample=True, 
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test it
print(ask_bot("Who created this website?"))